# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096930


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:12,  2.60s/it]

Rendering models:  10%|█         | 3/30 [00:06<01:01,  2.27s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:42,  1.63s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:26,  1.16s/it]

...

Rendering models:  80%|████████  | 24/30 [00:08<00:00,  7.23it/s]

Rendering models:  87%|████████▋ | 26/30 [00:08<00:00,  8.11it/s]

Rendering models:  93%|█████████▎| 28/30 [00:08<00:00,  8.31it/s]

Rendering models: 100%|██████████| 30/30 [00:08<00:00,  9.47it/s]

not-logged-in-3b0b929f4475fd276561    0.004691
not-logged-in-14800bc0cb17aef85ae6    0.006193
not-logged-in-3b0b929f4475fd276561    0.043598
Jnursssmith5263                       0.001717
ethanth                               0.001497
ellivc                                0.001785
boomod                                0.000469
24declanfm                            0.000582
whitelaughter                         0.000465
briebowman                            1.212473
DarioB                                0.067769
alphamoonbase                         0.001458
tominas                               0.000545
acapirala                             0.049351
Lavadude                              0.007081
not-logged-in-90294de164951e098d0b    0.000740
clars915                              0.001906
AspenTreez                            0.005941
kgenere                               0.091793
ywan3529                              0.011774
koby.sacramento                       0.415865
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())